In [1]:
import lightgbm as lgb
import joblib
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


In [2]:
# Load the dataset
tr_tr_new = joblib.load('../joblib/tr_tr_encoded.joblib')

In [3]:
te_tr_new = joblib.load('../joblib/te_tr_encoded.joblib')

In [4]:
# Splitting the dataset into features and target
X = tr_tr_new.drop(columns=['isFraud'])
y = tr_tr_new['isFraud']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Assuming model_lgbm is your trained LightGBM model and model_xgb is your trained XGBoost model
# And X_train, X_valid and y_train are your training and validation data

# Get the predictions from both models for your training data
model_lgbm = joblib.load('lightgbm_model[0.9534].joblib')
train_preds_lgbm = model_lgbm.predict(X_train)
model_xgb = joblib.load('../models_libjob/xgboost_model[0.9728].joblib')
train_preds_xgb = model_xgb.predict_proba(X_train)[:, 1]

J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dt

In [6]:
# Stack predictions together
stacked_train_predictions = np.column_stack((train_preds_lgbm, train_preds_xgb))

# Train a logistic regression model on the stacked predictions
meta_model = LogisticRegression().fit(stacked_train_predictions, y_train)

In [7]:
# Get predictions for test set (te_tr_new) and stack them
test_preds_lgbm = model_lgbm.predict(te_tr_new)
test_preds_xgb = model_xgb.predict_proba(te_tr_new)[:, 1]
stacked_test_predictions = np.column_stack((test_preds_lgbm, test_preds_xgb))

J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dt

In [8]:
# Make final predictions using the meta model
final_preds = meta_model.predict_proba(stacked_test_predictions)[:, 1]

# Create the output DataFrame
output_df = pd.DataFrame({
    'TransactionID': te_tr_new.reset_index()['TransactionID'],
    'isFraud': final_preds
})

# Save the DataFrame to a CSV file
output_df.to_csv('stacked_predictions.csv', index=False)
